## <span style="color: #0058bd;">Regression: Tree-Based Methods</span>

In [ ]:
# Imports
import numpy as np
import pandas as pd
from statsmodels.api import OLS
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

### <span style="color: #0058bd;">Exercise 56</span>

Load the portfolio tracking data and compute the in- and out-of-sample SSE for OLS.

In [ ]:
# Download and re-format VWM data (see 9_exercises_regression_subset_stepwise.ipynb)
vwm = pd.read_csv("../data/VWM.csv", index_col="Date")
vwm.index = pd.to_datetime(vwm.index, format="%Y%m")
vwm = vwm.resample("M").last()
# Download and re-format industry portfolio data (see 9_exercises_regression_subset_stepwise.ipynb)
industries = pd.read_csv("../data/12_Industry_portfolios.csv", index_col="Date")
industries.index = pd.to_datetime(industries.index, format="%Y%m")
industries = industries.resample("M").last()

In [ ]:
# Subset data to 1980-2014 (training set)
x = industries["1980":"2014"]
y = vwm["VWM"]["1980":"2014"]
t, p = x.shape

#### <span style="color: #0058bd;">Explanation</span>

We first show the OLS in-sample SSE as a benchmark value, and then its out-of-sample SSE.

In [ ]:
tss = y.T @ y
res = OLS(y, x).fit()
print(f"OLS SSE is {tss * (1-res.rsquared):0.2f}.")

In [ ]:
# Select the out-of-sample data
y_oos = vwm.loc["2015":, "VWM"]
x_oos = industries["2015":]

resid = y_oos - x_oos @ res.params
ols_oos_sse = resid.T @ resid
print(f"The out-of-sample SSE for OLS is {ols_oos_sse:0.2f}.")

### <span style="color: #0058bd;">Exercise 57</span>

Fit a default Random Forest in a reproducible manner to the portfolio tracking data and compute the in- and out-of-sample SSE.

**Warning**: This exercise is simply an example of how to use these methods. In general, tree-based models are terrible choices for tracking portfolio construction since the final model is not a weighted combination of the returns, but instead depends on non-linear transformation of the returns. This makes implementation of a tree-based estimator virtually impossible. 

#### <span style="color: #0058bd;">Explanation</span>
Random Forests fit ensembles of trees (combinations) using a random sample of the regressors in each.  Here we fit a default Random Forest where we use the $\sqrt{p}$ rule for feature selection within each tree.

The in-sample SSE is very good and much smaller than the in-sample SSE of OLS.

In [ ]:
rfr = RandomForestRegressor(max_features="sqrt", random_state=2309)
rfr = rfr.fit(x, y)
resid = y - rfr.predict(x)
print(f"The RandomForest SSE is {resid.T@resid:0.2f}.")

#### <span style="color: #0058bd;">Explanation</span>

The out-of-sample SSE, however, is quite a bit worse than OLS.  Tree-based models are not good models for tracking portfolio construction.

In [ ]:
pred = rfr.predict(x_oos)
resid = y_oos - pred
rf_oos_sse = resid.T @ resid
print(f"The out-of-sample SSE for the default RF is {rf_oos_sse:0.2f}.")

### <span style="color: #0058bd;">Exercise 58</span>

Optimize the key tuning parameters of the Random Forest using cross-validation and compute the out-of-sample SSE of the preferred model.

In [ ]:
parameters = {
    "n_estimators": [100, 250, 500, 1000],
    "max_features": [1.0, "sqrt"],
    "max_leaf_nodes": [50, 100, 200, 225, 250],
}

rfr = RandomForestRegressor(random_state=2309)
gscv = GridSearchCV(
    rfr, parameters, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1
)

gscv = gscv.fit(x, y)

#### <span style="color: #0058bd;">Explanation</span>

**Tuning Parameters Random Forests** 
- `max_leaf_nodes`: Limits the maximum number of terminal nodes (leaves) of a tree. Constraining the number of leaf nodes reduces tree complexity, which can help prevent overfitting. Smaller values lead to shallower trees, reducing computation times and variance (at the cost of underfitting).
- `max_depth`: Limits the depth of the tree i.e., the number of splits a tree can make from the root to the leaves. A deeper tree can fit the training data better but risks overfitting.
    - **Note:** `max_depth` limits the depth of the tree i.e., the number of levels from the root node to the furthest leaf, while `max_leaf_nodes` limits the total number of leafs (terminal nodes) in the tree, irrespective of how deep the tree grows.
- `n_estimators`: Determines the number of trees $M$ in the forest. A larger number of trees improves model performance by reducing variance (averaging many predictors). Beyond a certain point, adding more trees has diminishing returns.
- `max_features`: Specifies the number of features for the randomly selected subset when looking for the best split. Smaller values reduce overfitting as randomness is introduced into the tree-building process. Larger values (e.g., using all features) increase tree depth but may overfit. Lower values reduce overfitting and potentially decorrelate trees in the forest.
    - **Intuition:** If a dataset has a small number of highly predictive features, trees in the forest might keep splitting on those same features repeatedly. This makes trees correlated (less diverse), which reduces the benefit of averaging in the ensemble.
    - "auto" as default uses $\sqrt{\text{number of features}}$.
- `min_samples_split`: Minimum number of observations (samples) required in a node for it to be eligible for splitting. If the number of samples in a node is less than `min_sample_split`, that node becomes a leaf (terminal) node and no further splits are made. Small values allow nodes to split aggressively, even with very few observations, which results in deeper trees with smaller leaf nodes. However, it increases the risk of overfitting because the model can learn noise in the data.


`GridSearchCV` allows us to compute the cross-validated score of a model for a combination of input parameters. This method is similar to writing a number of loops across each of the parameters and then cross-validating the model for each distinct combination.  

The key input to `GridSearchCV` is a dictionary where the keys are model parameter names and the values are the values that should be considered in the search.  The model is then automatically cross-validated for all of combinations of the parameters. 

**Note**: This cell may run of an extended period, depending on your system.

The best estimator in the sense of minimizing the score function (negative MSE here) is available using the `best_estimator_` attribute. This is a `RandomForestRegressor` with the CV-optimized parameters. This estimator can then be fit to the data.

In [ ]:
rfr_best = gscv.best_estimator_.fit(x, y)
rfr_best

In [ ]:
resid = y - rfr_best.predict(x)
print(f"The in-sample SSE of the best model is {resid.T @ resid:0.1f}")

#### <span style="color: #0058bd;">Explanation</span>

The in-sample SSE is very good, and is slightly better than the naive attempt.

Note that the cross-validated sse is related to the negative MSE usign the relationship

$$ \text{Neg MSE} = -\frac{SSE_{xv}}{n} $$

The values are stored in a dictionary `gscv.cv_results_` using the key `"mean_test_score"`.  We can convert these to cross-validated SSE for comparison with other methods. These are all higher than what we saw with regression methods.

In [ ]:
sse_xv = -t * gscv.cv_results_["mean_test_score"]
sse_xv

#### <span style="color: #0058bd;">Explanation</span>

`cv_results_` also contains the parameters used in each configuration. Here we can build a `DataFrame` that examines the better parameterizations by merging these values with the $SSE_{xv}$ and sorting.  We see that the best configurations always used `"sqrt"` for `max_features`, and the 500 consistently outperformed 250 or 1000 estimators. 

In [ ]:
df = pd.DataFrame(gscv.cv_results_["params"])
df["sse_xv"] = sse_xv
df.sort_values("sse_xv").head(10)

#### <span style="color: #0058bd;">Explanation</span>

Finally we can compute the OOS SSE using the `predict` method with the out-of-sample data. This value is poor when compared to OLS.  This indicates (not surprisingly) that tree-based methods are not good ways to fit financial return data.

In [ ]:
pred = rfr_best.predict(x_oos)
resid = y_oos - pred
rf_oos_sse = resid @ resid
print(f"The out-of-sample SSE for the optimized RF is {rf_oos_sse:0.2f}")

### <span style="color: #0058bd;">Exercise 59</span>

Boosting is often a better alternative to Random Forests since it limits tree depth, and in turn, variable interactions. Fit a default boosted regression tree to the portfolio tracking data, and compute the out-of-sample SSE.

In [ ]:
gbr = GradientBoostingRegressor(random_state=2309)
gbr.fit(x, y)
pred = gbr.predict(x_oos)
resid = y_oos - pred
gbr_oos_sse = resid @ resid
gbr_oos_sse

#### <span style="color: #0058bd;">Explanation</span>

Here we fit a default boosted regression tree using `GradientBoostingRegressor`.  It is always a good idea to set `random_state` to ensure results are reproducible. We compute the OOS SSE and see that the default parameters perform well when compared to either Random Forest.

### <span style="color: #0058bd;">Exercise 60</span>

Optimize the key parameters of the boosted regression tree using cross-validation.

In [ ]:
parameters = {
    "learning_rate": [0.01, 0.025, 0.05, 0.1, 0.2],
    "n_estimators": [1000, 2000, 4000, 8000, 12000],
    "max_leaf_nodes": [2, 3, 4, 6],
}

gscv = GridSearchCV(
    gbr, parameters, n_jobs=-1, scoring="neg_mean_squared_error", verbose=1
)
gscv.fit(x, y)

#### <span style="color: #0058bd;">Explanation</span>

Boosted models can be tuned like any other approach.  Here we use `GridSearchCV` again to search for good choices of the learning rate ($\lambda$ in the notes), the number of estimators ($B$ in the notes), and the `max_leaf_nodes` ($d$ in the notes).

**Note**: This cell can take a while to run, depending on your machine.

The preferred configuration has a large number of estimators with a relatively low learning rate and small trees.

In [ ]:
best_gbr = gscv.best_estimator_.fit(x, y)
best_gbr

#### <span style="color: #0058bd;">Explanation</span>

When we look at the top performing estimators, we see that small trees combined, slow learning and many estimators consistently perform best.

In [ ]:
sse_xv = -t * gscv.cv_results_["mean_test_score"]
df = pd.DataFrame(gscv.cv_results_["params"])
df["sse_xv"] = sse_xv
df = df.sort_values("sse_xv")
df.index = np.arange(1, df.shape[0] + 1)
df.head(10)

### <span style="color: #0058bd;">Aside: High-Level Comparison Random Forest and Gradient Boosting Regressor</span>

Both Random Forest (RF) and Gradient Boosting (GB) are so-called ensemble learning technique that build multiple decision trees for prediction. The main difference between both is how trees are constructed and combined to make predictions.

##### <span style="color: #0058bd;">Random Forest Regressor</span>
The RF procedure builds multiple decision trees independently and combines their outputs - averages in regression tasks. The central concept is **bagging (bootstrap aggregating)**, where:
- Each tree is trained on a random subset (with replacement) of the training data;
- A random subset of features is considered at each split to reduce the correlation between trees.

Let $\{ T_1, T_2, \dots, T_M \}$ be $M$ decision trees, each trained on a random subset of the data. The prediction for a new sample $X$ is the average of the predictions from all trees
$$
    \hat{Y} = \frac{1}{M} \sum_{i=1}^M T_i(X).
$$
The **goal** is variance reduction by combining multiple week learners (decision trees).


##### <span style="color: #0058bd;">Gradient Boosting Regressor</span>

The GF procedure builds trees sequentially, where each tree corrects the residual errors of the previous tree. The model minimizes a specified loss function (e.g., MSE for regression) by adding trees that focus on reducing the residuals. 

GB starts with an initial prediction $\hat{Y}_0$, often the mean of the target $Y$. At each iteration $m$, a tree $T_m(X)$ is built to predict the residuals (negative gradient of the loss function with respect to the current predictions)
$$
    r_m = - \frac{ \partial \text{Loss}(Y, \hat{Y}_{m-1})}{\partial \hat{Y}_{m-1}}.
$$
For regression with MSE as loss $r_m = Y - \hat{Y}_{m-1}$. The decision tree at iteration $m$ will try to predict the new target $r_m$. The residuals are the mechanism by which GB learns incrementally. Focusing on the part of the target not yet explained, the model efficiently reduces the bias at each step.

Once the tree $T_m(X)$ is built on $r_m$, the model updates the predictions by adding the weighted contributions of the new tree
$$
    \hat{Y}_m = \hat{Y}_{m-1} + \eta \cdot T_m(X),
$$
where $\eta$ is the learning rate that controls the step size, or in other words, how much the new tree contributes to the updated prediction. As such, choosing a lower learning rate $\eta$ requires more trees to converge. However, it also reduces the risk of overfitting by ensuring that each tree makes only a small contribution. The final prediction combines all trees
$$
    \hat{Y} = \sum_{m=1}^M \eta \cdot T_m(X).
$$

The **goal** is reduction of bias and variance by iteratively improving the model.


##### <span style="color: #0058bd;">Gradient Boosting Over Random Forests</span>

GB is the better choice in certain situations due to the inherent properties.
- Direct optimization of the loss function makes it better at reducing bias in the model. This is particularly useful when the data has complex patterns.
- Custom loss function can be tailored to specific tasks (e.g., asymmetric errors or outlier handling).
- By sequentially correcting residuals, GB effectively focuses on the "hard-to-predict" cases, improving model performance for difficult tasks.

GB is the better choice in following scenarios.
- High bias problems: GB iteratively reduces bias (RF focuses on variance).
- Custom errors: GB allows to customize the loss function.
- Smaller datasets: When tuned, GB performs better on smaller datasets.
- Feature interactions: GB captures complex interactions better through sequential trees.
- Outliers: By choosing a robust loss function, GB can handle outliers better.

**Summary:** If (i) your dataset is large and noisy, (ii) computational efficiency is a priority of yours, and/or (iii) you need a model with low sensitivity to hyperparameters, RFs may be your choice. If (i) you are working with smaller datasets or datasets with complex patterns, (ii) reducing bias is critical, and/or (iii) optimize for a specific custom loss, GB may be your choice. 


#### <span style="color: #0058bd;">Summary of High Level Differences </span>

|                | **Random Forests**                                  | **Gradient Boosting**                                  |
|--------------------------|-----------------------------------------------------------|---------------------------------------------------------------|
| **Tree Construction**     | Trees built **independently** using bootstrap samples     | Trees built **sequentially**, correcting previous errors      |
| **Aggregation**           | Combines tree outputs via averaging                      | Adds tree outputs iteratively to minimize a loss function     |
| **Bias vs. Variance**      | Primarily reduces **variance** through averaging         | Reduces both **bias and variance** iteratively                |
| **Training Speed**        | Faster, as trees are built independently                 | Slower, due to sequential tree building                      |
| **Overfitting Risk**      | Lower risk of overfitting due to averaging               | Higher risk, but can be controlled with regularization        |
| **Hyperparameter Sensitivity** | Less sensitive; works well with default parameters    | More sensitive; requires careful tuning (e.g., learning rate, tree depth) |
| **Interpretability**      | Moderate interpretability (feature importance is averaged)| Less interpretable, as predictions depend on sequential corrections |


### <span style="color: #0058bd;">Exercise 61</span>

Compute the out-of-sample SSE for the selected boosted regression tree.

In [ ]:
pred = best_gbr.predict(x_oos)
resid = y_oos - pred
rf_oos_sse = resid @ resid
rf_oos_sse

#### <span style="color: #0058bd;">Explanation</span>

We can generate the out-of-sample SSE for the optimized GBR. We see that while it is substantially improved over what we found with a Regression Tree, it is still 15% worse then what plain OLS achieves.